In [59]:
# ref: https://machinelearningmastery.com/the-attention-mechanism-from-scratch/
from numpy import array



In [60]:
# encoder representations of four different words
word_1 = array([1, 0, 0])
word_2 = array([0, 1, 0])
word_3 = array([1, 1, 0])
word_4 = array([0, 0, 1])

In [61]:
from numpy import random
# generating the weight matrices
random.seed(42) # to allow us to reproduce the same attention values
W_Q = random.randint(3, size=(3, 3))
W_K = random.randint(3, size=(3, 3))
W_V = random.randint(3, size=(3, 3))

In [62]:
...
# generating the queries, keys and values
query_1 = word_1 @ W_Q
key_1 = word_1 @ W_K
value_1 = word_1 @ W_V

query_2 = word_2 @ W_Q
key_2 = word_2 @ W_K
value_2 = word_2 @ W_V

query_3 = word_3 @ W_Q
key_3 = word_3 @ W_K
value_3 = word_3 @ W_V

query_4 = word_4 @ W_Q
key_4 = word_4 @ W_K
value_4 = word_4 @ W_V

In [63]:
from numpy import dot
# scoring the first query vector against all key vectors
scores = array([dot(query_1, key_1), dot(query_1, key_2), dot(query_1, key_3), dot(query_1, key_4)])

In [42]:
scores,query_1, key_3,query_1, key_4

(array([ 8,  2, 10,  2]),
 array([2, 0, 2]),
 array([2, 4, 3]),
 array([2, 0, 2]),
 array([0, 1, 1]))

In [90]:
# computing the weights by a softmax operation
import torch
tscores = torch.tensor([scores]).float()
m = torch.nn.Softmax(dim=1)
weights= m(tscores)
weights=weights[0][:]

In [91]:
# computing the attention by a weighted sum of the value vectors
attention = (weights[0] * value_1) + (weights[1] * value_2) + (weights[2] * value_3) + (weights[3] * value_4)
print(attention)

tensor([0.9994, 1.8800, 0.8806], dtype=torch.float64)


In [115]:
from numpy import array
from numpy import random
from numpy import dot
from scipy.special import softmax

# encoder representations of four different words
word_1 = array([1, 0, 0])
word_2 = array([0, 1, 0])
word_3 = array([1, 1, 0])
word_4 = array([0, 0, 1])

# stacking the word embeddings into a single array
words = array([word_1, word_2, word_3, word_4])

# generating the weight matrices
random.seed(42)
W_Q = random.randint(3, size=(3, 3))
W_K = random.randint(3, size=(3, 3))
W_V = random.randint(3, size=(3, 3))

# generating the queries, keys and values
Q = words @ W_Q #query #  st_1: previous decoder output  st-1      
K = words @ W_K #key#hi#  hi, encoded  hidden state. K.shape is [4,3]. since [4,3]*[3,3]
V = words @ W_V #values#  hi

# scoring the query vectors against all key vectors
scores = Q @ K.transpose() # alignment scores K.transpose().shape is [3,4]
###########My dscoveries###############
'''
Notation: 
a: length of words sequence, in this case:4. 
b: length of each word. In this case:3
c: row length of W_Q,W_K,W_V
d: colum length of W_Q,W_K,W_V

Discoveries 0: row lenth of W_Q,W_K,W_V always == b, 
               so b==c

Discoveries 1: scores is  always a square matrix, with shape a*a
    Why:   words.shape: a*b, W_Q.shape: c*d
        so     :Q.shape: a*d, K.shape: a*d, K_transpose().shape d*a
        so     :Q.shape*K_transpose().shape = a*a
        finally:scores.shape==a*a

'''
#######################################

# computing the weights by a softmax operation
weights = softmax(scores / K.shape[1] ** 0.5, axis=1)

# computing the attention by a weighted sum of the value vectors
attention = weights @ V
###########My dscoveries###############
'''
Notation: 
a: length of words sequence, in this case:4. 
b: length of each word. In this case:3
c: row length of W_Q,W_K,W_V
d: colum length of W_Q,W_K,W_V

Discoveries 2: attention is  always a matrix with shape a*d
    Why:   words.shape: a*b, W_V.shape: c*d
        so     :V.shape: a*d, scores.shape==weights.shape: a*a
        so     :weights.shape*V.shape = a*d
        finally:attention.shape==a*d
'''
#######################################
print(attention)
print(scores)

[[0.98522025 1.74174051 0.75652026]
 [0.90965265 1.40965265 0.5       ]
 [0.99851226 1.75849334 0.75998108]
 [0.99560386 1.90407309 0.90846923]]
[[ 8  2 10  2]
 [ 4  0  4  0]
 [12  2 14  2]
 [10  4 14  3]]


In [99]:
weights = softmax(scores / K.shape[1] ** 0.5, axis=1)

array([[2.36089863e-01, 7.38987555e-03, 7.49130386e-01, 7.38987555e-03],
       [4.54826323e-01, 4.51736775e-02, 4.54826323e-01, 4.51736775e-02],
       [2.39275049e-01, 7.43870015e-04, 7.59237211e-01, 7.43870015e-04],
       [8.99501754e-02, 2.81554063e-03, 9.05653685e-01, 1.58059922e-03]])

(3, 3)